In [1]:
! pip install datasets

     |████████████████████████████████| 342 kB 25.8 MB/s 
     |████████████████████████████████| 1.1 MB 56.9 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 212 kB 70.8 MB/s 
     |████████████████████████████████| 136 kB 74.5 MB/s 
     |████████████████████████████████| 127 kB 75.1 MB/s 
     |████████████████████████████████| 271 kB 59.5 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 77.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
! pip install transformers

     |████████████████████████████████| 4.2 MB 29.0 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
     |████████████████████████████████| 6.6 MB 56.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split, GroupShuffleSplit, LeaveOneGroupOut, KFold, StratifiedKFold
from datasets import Dataset
from ast import literal_eval
from transformers import EarlyStoppingCallback

In [5]:
class Dataset_info:
    def __init__(self, dataset_name):
        self.dataset_name = dataset_name
dataset_info_dict = {
    "lcc": Dataset_info("lcc_lcc_lcc"),
    "yulia": Dataset_info("yulia_yulia_yulia"),
    "wiktionary": Dataset_info("wiktionary_wiktionary_wiktionary"),
    "lcc_yulia": Dataset_info("lcc_lcc_yulia"),
    "lcc_wiktionary": Dataset_info("lcc_lcc_wiktionary"),
    "yulia_lcc": Dataset_info("yulia_yulia_lcc"),
    "yulia_wiktionary": Dataset_info("yulia_yulia_wiktionary"),
    "wiktionary_lcc": Dataset_info("wiktionary_wiktionary_lcc"),
    "wiktionary_yulia": Dataset_info("wiktionary_wiktionary_yulia"),
}

In [50]:
my_dataset_info = dataset_info_dict['wiktionary']

In [7]:
check = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(check)

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [51]:
class Dataset_handler:
    def __init__(self, dataset_info: Dataset_info):
        self.dataset = datasets.DatasetDict()
        self.tokenized_dataset = None
        self.dataset_info = dataset_info
        self.labels_list = None
        # CACHE
        self.global_cache_counter = 0
        self.cache_last_hashable_input = ""
        self.columns_to_return = ['input_ids', 'label', 'attention_mask']
        
        if dataset_info.dataset_name == "lcc_lcc_lcc":
            frac = 1
            self.csv_to_dataset('./drive/MyDrive/metaphor-psycho/plm probing/ru/train.csv', data_type="train", fraction = frac)
            self.csv_to_dataset('./drive/MyDrive/metaphor-psycho/plm probing/ru/dev.csv', data_type="dev", fraction = frac)
            self.csv_to_dataset('./drive/MyDrive/metaphor-psycho/plm probing/ru/test.csv', data_type="test", fraction = frac)

        elif dataset_info.dataset_name == "yulia_yulia_yulia":
            frac = 1
            self.yulia_to_dataset('./drive/MyDrive/metaphor-psycho/verbs_with_wfs_full.csv', data_type="all", fraction = frac)
        elif dataset_info.dataset_name == "wiktionary_wiktionary_wiktionary":
            frac = 1
            self.yulia_to_dataset('./drive/MyDrive/rscf/wiktionary/wiktionary_verb_examples_full.csv', data_type="all", fraction = frac)
        elif dataset_info.dataset_name == "manual":
            frac = 1
            f = open("./manual_dataset.json", "w")
            f.write('{"text": "' + dataset_info.manual_text + '", "targets": [{"span1": [0, 0], "label": "' + my_dataset_handler.labels_list[0] + '"}]}')
            f.write('\n{"text": "' + dataset_info.manual_text + '", "targets": [{"span1": [0, 0], "label": "' + my_dataset_handler.labels_list[1] + '"}]}')
            f.close()
            self.json_to_dataset('./manual_dataset.json', data_type="train", fraction = frac, to_sentence_span=True)
            self.json_to_dataset('./manual_dataset.json', data_type="dev", fraction = frac, to_sentence_span=True)
            self.json_to_dataset('./manual_dataset.json', data_type="test", fraction = frac, to_sentence_span=True)
        else:
            #throw("Error: Unkown dataset name!")
            infos = dataset_info.dataset_name.split('_')
            frac=1
            for i, fold in enumerate(['train', 'dev', 'test']):
              if infos[i] == 'lcc':
                self.csv_to_dataset('./drive/MyDrive/metaphor-psycho/plm probing/ru/' + fold +'.csv', data_type=fold, fraction = frac)
              elif infos[i] == 'yulia':
                self.yulia_to_dataset('./drive/MyDrive/metaphor-psycho/verbs_with_wfs_full.csv', data_type=fold, fraction = frac)
              elif infos[i] == 'wiktionary':
                self.yulia_to_dataset('./drive/MyDrive/rscf/wiktionary/wiktionary_verb_examples_full.csv', data_type=fold, fraction = frac)

        print("⌛ Tokenizing Dataset")
        self.tokenized_dataset = self.preprocess(self.dataset)
        # self.tokenized_dataset = self.tokenize_dataset(self.dataset)
        # print("⌛ Adding One Hot Representation of Labels")
        # self.tokenized_dataset = self.one_hot_dataset_labels(self.tokenized_dataset)
    
    def csv_to_dataset(self, csv_path, data_type="train", fraction=1):
      df = pd.read_csv(csv_path,converters={"span1": literal_eval})
      df['label'] = df.label.apply(lambda x: 1 if x =='Metaphor' else 0)
      df['wordform'] = df.apply(lambda x: get_wordform(x.text, x.span1), axis=1)
      df['text2'] = df.wordform.apply(gettext2)
      self.dataset[data_type] = datasets.Dataset.from_pandas(df[['text', 'text2', 'label']])
      return self.dataset

    def yulia_to_dataset(self, csv_path, data_type="train", fraction=1):
      df = pd.read_csv(csv_path, converters={"span1": literal_eval}, index_col=0)
      df['label'] = df.Class
      if not ('text' in df.columns):
        df['text'] = df['sent']
      df['fold'] = df.fold.apply(lambda x: 'dev' if x == 'val' else x )
      df['text2'] = df.wordforms.apply(gettext2)
      if data_type=="all":
        for data_type in ['train', 'dev', 'test']:
          self.dataset[data_type] = datasets.Dataset.from_pandas(df[['text', 'text2', 'label']][df.fold == data_type])
      else:
          self.dataset[data_type] = datasets.Dataset.from_pandas(df[['text', 'text2', 'label']][df.fold == data_type])
      
      return self.dataset

    def preprocess(self, dataset):        
      tokenized_dataset = dataset.map(preprocess_function,
                                      fn_kwargs={"tokenizer": tokenizer},
                                      batched = True)
      
      tokenized_dataset.set_format(type='torch', columns=self.columns_to_return)
      return tokenized_dataset

def get_wordform(text, span1):
        try:
          return text.split()[span1[0]]
        except:
          print(span1, text)    
def gettext2(v):
      v = v.split(',')[0].strip('[]\' ')
      verb = v[0].upper() + v[1:]
      #return verb + ' - это метафора .'
      return '\'' + verb + '\' - это метафора .'

def preprocess_function(examples, **fn_kwargs):
      tokenizer = fn_kwargs["tokenizer"]
      return tokenizer(examples['text'], examples['text2'], truncation=True, max_length=128, padding = True)

In [60]:
my_dataset_info = dataset_info_dict['yulia']
my_dataset_handler = Dataset_handler(my_dataset_info)
rnd_idx = np.random.randint(1000)
# rnd_idx = 58000
part = "train"
display(pd.DataFrame(my_dataset_handler.tokenized_dataset['train'])[0:3])
display(pd.DataFrame(my_dataset_handler.tokenized_dataset['test'])[0:3])
print("idx =", rnd_idx)
print(my_dataset_handler.tokenized_dataset)
print("Original Spans:", my_dataset_handler.dataset[part][rnd_idx])
print("Tokenized Spans:", my_dataset_handler.tokenized_dataset[part][rnd_idx])

⌛ Tokenizing Dataset


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

,label,input_ids,attention_mask
0,tensor(1),"[tensor(101), tensor(27995), tensor(3629), ten...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
1,tensor(1),"[tensor(101), tensor(29022), tensor(106775), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
2,tensor(1),"[tensor(101), tensor(7974), tensor(14829), ten...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."


,label,input_ids,attention_mask
0,tensor(0),"[tensor(101), tensor(156), tensor(2886), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
1,tensor(0),"[tensor(101), tensor(19438), tensor(880), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
2,tensor(0),"[tensor(101), tensor(28942), tensor(39883), te...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."


idx = 435
DatasetDict({
    train: Dataset({
        features: ['text', 'text2', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5149
    })
    dev: Dataset({
        features: ['text', 'text2', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 607
    })
    test: Dataset({
        features: ['text', 'text2', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1412
    })
})
Original Spans: {'text': 'Дальше « доить » и « разводить » можно только совсем запущенных « потребителей » .', 'text2': "'Доить' - это метафора .", 'label': 1, '__index_level_0__': 435}
Tokenized Spans: {'label': tensor(1), 'input_ids': tensor([  101, 21006,   304,  2785,  2603,   326,   851,   304, 25883,  2603,
          326,  7021,  4564, 20470, 30075,  1755,   304,  9829, 43152,   852,
          326,   132,   102,   118,  2785,  2603,   118,   130,  3998, 65593,
     

In [54]:
model = AutoModelForSequenceClassification.from_pretrained(check)
metric_name = "f1"
args = TrainingArguments(
      f"metaphor-verbs",
      evaluation_strategy = "epoch",
      save_strategy = "epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=10,
      weight_decay=0.01,
      load_best_model_at_end=True,
      metric_for_best_model=metric_name,
      push_to_hub=False,
  )
def compute_metrics(pred):
      labels = pred.label_ids
      preds = pred.predictions.argmax(-1)
      #preds = pred.predictions
      precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
      acc = accuracy_score(labels, preds)
      return {
          'accuracy': acc,
          'f1': f1,
          'precision': precision,
          'recall': recall
      }
trainer = Trainer(
      model,
      args,
      train_dataset=my_dataset_handler.tokenized_dataset['train'],
      eval_dataset=my_dataset_handler.tokenized_dataset['dev'],
      tokenizer=tokenizer,
      compute_metrics=compute_metrics,
      callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
  )

loading configuration file https://huggingface.co/cointegrated/rubert-base-cased-nli-twoway/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/13d362532a4bb0918b7de7be610be12a6e048b1ddf64d02466844531960612bb.5002f0c71cd5244db4073d30a5b3871a6b0bb0dcac24ceaf87312cc8ba3e56cc
Model config BertConfig {
  "_name_or_path": "cointegrated/rubert-base-cased-nli-twoway",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "not_entailment",
    "1": "entailment"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "entailment": 1,
    "not_entailment": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "out

In [55]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, text2. If text, __index_level_0__, text2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3318
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2080


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.621693,0.674468,0.634845,0.573276,0.711230
2,No log,0.642880,0.682979,0.575499,0.615854,0.540107
3,0.478900,1.216657,0.685106,0.519481,0.661157,0.427807
4,0.478900,1.788817,0.657447,0.538682,0.580247,0.502674
5,0.132200,2.122893,0.661702,0.461017,0.629630,0.363636
6,0.132200,2.357243,0.672340,0.536145,0.613793,0.475936


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, text2. If text, __index_level_0__, text2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 470
  Batch size = 16
Saving model checkpoint to metaphor-verbs/checkpoint-208
Configuration saved in metaphor-verbs/checkpoint-208/config.json
Model weights saved in metaphor-verbs/checkpoint-208/pytorch_model.bin
tokenizer config file saved in metaphor-verbs/checkpoint-208/tokenizer_config.json
Special tokens file saved in metaphor-verbs/checkpoint-208/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, text2. If text, __index_level_0__, text2 are not expected by `BertForSequenceClassificatio

TrainOutput(global_step=1248, training_loss=0.2501657689229036, metrics={'train_runtime': 546.157, 'train_samples_per_second': 60.752, 'train_steps_per_second': 3.808, 'total_flos': 1309503722526720.0, 'train_loss': 0.2501657689229036, 'epoch': 6.0})

In [56]:
trainer.evaluate(my_dataset_handler.tokenized_dataset['test'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, text2. If text, __index_level_0__, text2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 952
  Batch size = 16


{'epoch': 6.0,
 'eval_accuracy': 0.7132352941176471,
 'eval_f1': 0.6539923954372624,
 'eval_loss': 0.5702189207077026,
 'eval_precision': 0.6013986013986014,
 'eval_recall': 0.7166666666666667,
 'eval_runtime': 7.0124,
 'eval_samples_per_second': 135.759,
 'eval_steps_per_second': 8.556}

In [57]:
trainer.evaluate(my_dataset_handler.tokenized_dataset['dev'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, text2. If text, __index_level_0__, text2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 470
  Batch size = 16


{'epoch': 6.0,
 'eval_accuracy': 0.674468085106383,
 'eval_f1': 0.6348448687350835,
 'eval_loss': 0.6216930150985718,
 'eval_precision': 0.5732758620689655,
 'eval_recall': 0.7112299465240641,
 'eval_runtime': 2.7056,
 'eval_samples_per_second': 173.713,
 'eval_steps_per_second': 11.088}

In [61]:
trainer.evaluate(my_dataset_handler.tokenized_dataset['test'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__, text2. If text, __index_level_0__, text2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1412
  Batch size = 16


{'epoch': 6.0,
 'eval_accuracy': 0.7606232294617564,
 'eval_f1': 0.7578796561604585,
 'eval_loss': 0.5104361176490784,
 'eval_precision': 0.7655571635311144,
 'eval_recall': 0.750354609929078,
 'eval_runtime': 9.7173,
 'eval_samples_per_second': 145.307,
 'eval_steps_per_second': 9.159}